# Prediction of Premier League Results

In [1]:
import pandas as pd
from datetime import datetime
pd.set_option('max_columns', None)

In [2]:
# read and join teams' stats

seasons = ['1718','1819','1920','2021']
full_teamdata = []
best_player = pd.read_csv('poty.csv')
for season in seasons:
    table = pd.read_csv(season + 'table.csv')
    forteam = pd.read_csv(season + 'for.csv')
    against = pd.read_csv(season + 'against.csv')
    shot = pd.read_csv(season + 'shot.csv')
    season_full = pd.concat([table,forteam,against,shot],axis=1)
    season_correct = "20" + season[:2] + "-" + season[2:]
    season_full['league'] = season_correct
    season_full['Best Player'] = best_player.dropna(subset=[season_correct]).reset_index()[season_correct]
    full_teamdata.append(season_full)
full_teamtable = pd.concat(full_teamdata,ignore_index=True)

In [3]:
# clean teams' stats

full_teamtable = full_teamtable.drop("MP", axis=1)
full_teamtable = full_teamtable.drop("Attendance", axis=1)

top_scorer = full_teamtable['Top Team Scorer'].str.split(' - ', 1, expand=True)
full_teamtable['Top Scorer'] = top_scorer[0]
full_teamtable['Top Scorer Goals'] = pd.to_numeric(top_scorer[1])
full_teamtable = full_teamtable.drop('Top Team Scorer', axis=1)
full_teamtable = full_teamtable.rename(columns={"Squad":"team"})

full_teamtable.head()

,Rk,team,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Goalkeeper,# Pl,Age,Poss,Gls,Ast,G-PK,PK,PKatt,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,league,Best Player,Top Scorer,Top Scorer Goals
0,6,Arsenal,19,6,13,74,51,23,63,68.4,50.6,17.8,0.47,Petr Čech,30,26.8,61.5,73,61,69,4,5,1.92,1.61,3.53,1.82,3.42,68.4,64.6,51.3,115.9,1.80,1.35,3.15,1.70,3.05,51,38,46,5,6,52,4,1.34,1.00,2.34,1.21,2.21,50.6,46.1,34.0,80.1,1.33,0.89,2.23,1.21,2.11,590,227,38.5,15.53,5.97,0.12,0.30,17.4,30,0.11,4.6,4.4,2017-18,Aaron Ramsey,Alexandre Lacazette,14
1,12,Bournemouth,11,11,16,45,61,-16,44,44.8,61.4,-16.6,-0.44,Asmir Begović,22,26.9,47.8,45,36,42,3,3,1.18,0.95,2.13,1.11,2.05,44.8,42.6,32.2,74.8,1.18,0.85,2.03,1.12,1.97,60,44,58,2,3,73,1,1.58,1.16,2.74,1.53,2.68,61.4,59.3,43.7,103.0,1.62,1.15,2.77,1.56,2.71,473,153,32.3,12.45,4.03,0.09,0.27,17.5,13,0.09,0.2,-0.6,2017-18,Nathan Ake,Callum Wilson Joshua King,8
2,15,Brighton,9,13,16,34,54,-20,40,37.2,52.6,-15.5,-0.41,Mathew Ryan,24,27.5,43.7,33,25,28,5,7,0.87,0.66,1.53,0.74,1.39,37.2,31.8,23.0,54.8,0.98,0.60,1.58,0.84,1.44,49,36,48,1,3,30,2,1.29,0.95,2.24,1.26,2.21,52.6,50.3,34.5,84.9,1.38,0.91,2.29,1.32,2.23,381,113,29.7,10.03,2.97,0.07,0.25,17.9,3,0.09,-4.2,-3.8,2017-18,Pascal Groß,Glenn Murray,12
3,7,Burnley,14,12,12,36,39,-3,54,34.5,51.7,-17.3,-0.45,Nick Pope,24,27.5,43.9,35,26,35,0,0,0.92,0.68,1.61,0.92,1.61,34.5,34.5,23.8,58.3,0.91,0.63,1.53,0.91,1.53,38,29,35,3,5,45,6,1.00,0.76,1.76,0.92,1.68,51.7,48.2,37.4,85.7,1.36,0.99,2.35,1.27,2.25,387,126,32.6,10.18,3.32,0.09,0.28,16.0,16,0.09,0.5,0.5,2017-18,Nick Pope,Chris Wood,10
4,5,Chelsea,21,7,10,62,38,24,70,57.9,35.3,22.7,0.60,Thibaut Courtois,26,26.7,55.4,60,42,57,3,3,1.58,1.11,2.68,1.50,2.61,57.9,55.7,42.2,97.9,1.52,1.11,2.64,1.46,2.58,36,26,34,2,2,75,1,0.95,0.68,1.63,0.89,1.58,35.3,33.8,25.4,59.1,0.93,0.67,1.60,0.89,1.56,602,209,34.7,15.84,5.50,0.09,0.27,17.4,19,0.09,2.1,1.3,2017-18,N'Golo Kanté,Eden Hazard,12


In [4]:
# read and clean matches' stats
matches = pd.read_csv('matches.csv')
promoted1819 = ["Wolves","Cardiff City","Fulham"]
promoted1920 = ["Norwich City","Sheffield Utd","Aston Villa"]
promoted2021 = ["Leeds United", "West Bromwich Albion","Fulham"]
promoted2122 = ["Norwich City","Watford","Brentford"]
#match season names to team table names
matches.loc[matches.league == "2018/2019 English Premier League","league"] = "2017-18"
matches.loc[matches.league == "2019-20 English Premier League","league"] = "2018-19"
matches.loc[matches.league == "2020-21 English Premier League","league"] = "2019-20"
matches.loc[matches.league == "2021-22 English Premier League","league"] = "2020-21"
#match team names to team table names
matches.loc[matches.home == "Brighton & Hove Albion","home"] = "Brighton"
matches.loc[matches.away == "Brighton & Hove Albion","away"] = "Brighton"
matches.loc[matches.home == "Huddersfield Town","home"] = "Huddersfield"
matches.loc[matches.away == "Huddersfield Town","away"] = "Huddersfield"
matches.loc[matches.home == "AFC Bournemouth","home"] = "Bournemouth"
matches.loc[matches.away == "AFC Bournemouth","away"] = "Bournemouth"
matches.loc[matches.home == "Wolverhampton Wanderers","home"] = "Wolves"
matches.loc[matches.away == "Wolverhampton Wanderers","away"] = "Wolves"
matches.loc[matches.home == "Sheffield United","home"] = "Sheffield Utd"
matches.loc[matches.away == "Sheffield United","away"] = "Sheffield Utd"
matches.loc[matches.home == "Newcastle United","home"] = "Newcastle Utd"
matches.loc[matches.away == "Newcastle United","away"] = "Newcastle Utd"
matches.loc[matches.home == "Manchester United","home"] = "Manchester Utd"
matches.loc[matches.away == "Manchester United","away"] = "Manchester Utd"
matches.loc[matches.home == "West Ham United","home"] = "West Ham"
matches.loc[matches.away == "West Ham United","away"] = "West Ham"
matches.loc[matches.home == "Tottenham Hotspur","home"] = "Tottenham"
matches.loc[matches.away == "Tottenham Hotspur","away"] = "Tottenham"

In [5]:
matches = matches[matches["league"].isin(["2017-18", "2018-19","2019-20","2020-21"])]
matches.reset_index(inplace=True)
seasons = ['1819','1920','2021','2122']
all_odds = []
for season in seasons:
    odds = pd.read_csv(season +'odds.csv').loc[:,['AvgH','AvgD','AvgA','Avg>2.5','Avg<2.5']]
    all_odds.append(odds)
all_odds = pd.concat(all_odds,ignore_index=True)
matches = pd.concat([matches,all_odds],axis=1)
matches = matches[((matches["league"] == "2017-18") & (~matches["home"].isin(promoted1819)) & (~matches["away"].isin(promoted1819))) | ((matches["league"] == "2018-19") & (~matches["home"].isin(promoted1920)) & (~matches["away"].isin(promoted1920))) | ((matches["league"] == "2019-20") & (~matches["home"].isin(promoted2021)) & (~matches["away"].isin(promoted2021))) | ((matches["league"] == "2020-21") & (~matches["home"].isin(promoted2122)) & (~matches["away"].isin(promoted2122)))]
matches = matches.fillna(0)
matches = matches.rename(columns={"time (utc)":"time"})

month_and_day = matches['date'].apply(lambda date: date.split(", ")[1]).str.split(' ', 1, expand=True)
matches['day'] = month_and_day[1]
matches['day'] = pd.to_numeric(matches['day'])
matches['month'] = month_and_day[0].apply(lambda month: int(datetime.strptime(month, '%B').strftime('%m')))
matches = matches.drop('date', axis=1)

display(matches)

,index,home,away,year,time,attendance,league,home_score,away_score,home_starting_1,home_starting_2,home_starting_3,home_starting_4,home_starting_5,home_starting_6,home_starting_7,home_starting_8,home_starting_9,home_starting_10,home_starting_11,away_starting_1,away_starting_2,away_starting_3,away_starting_4,away_starting_5,away_starting_6,away_starting_7,away_starting_8,away_starting_9,away_starting_10,away_starting_11,AvgH,AvgD,AvgA,Avg>2.5,Avg<2.5,day,month
0,6462,Manchester Utd,Leicester City,2018,19:00,74439.0,2017-18,2,1,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Matteo Darmian,Andreas Pereira,Paul Pogba,Fred,Marcus Rashford,Alexis Sánchez,Juan Mata,Kasper Schmeichel,Harry Maguire,Wes Morgan,Ben Chilwell,Daniel Amartey,James Maddison,Wilfred Ndidi,Adrien Silva,Kelechi Iheanacho,Demarai Gray,Ricardo Pereira,1.56,3.92,7.06,2.03,1.79,10,8
1,6463,Newcastle Utd,Tottenham,2018,11:30,51749.0,2017-18,1,2,Martin Dubravka,Ciaran Clark,Jamaal Lascelles,Paul Dummett,DeAndre Yedlin,Mohamed Diamé,Jonjo Shelvey,Kenedy,Matt Ritchie,Joselu,Ayoze Perez,Hugo Lloris,Jan Vertonghen,Davinson Sánchez,Ben Davies,Serge Aurier,Eric Dier,Dele Alli,Moussa Sissoko,Harry Kane,Christian Eriksen,Lucas Moura,1.88,3.53,4.37,1.98,1.83,11,8
4,6466,Huddersfield,Chelsea,2018,14:00,24121.0,2017-18,0,3,Ben Hamer,Christopher Schindler,Terence Kongolo,Mathias Jørgensen,Phillip Biling,Aaron Mooy,Jonathan Hogg,Chris Löwe,Florent Hadergjonaj,Steve Mounié,Alex Pritchard,Kepa Arrizabalaga,David Luiz,Antonio Rüdiger,Marcos Alonso,César Azpilicueta,Jorginho,Ross Barkley,N'Golo Kanté,Álvaro Morata,Willian,Pedro,3.83,3.40,2.05,2.01,1.81,11,8
5,6467,Watford,Brighton,2018,14:00,20051.0,2017-18,2,0,Ben Foster,Christian Kabasele,Craig Cathcart,José Holebas,Daryl Janmaat,Étienne Capoue,Abdoulaye Doucouré,Troy Deeney,Andre Gray,Roberto Pereyra,Will Hughes,Mat Ryan,Lewis Dunk,Shane Duffy,Bernardo,Bruno,Davy Propper,Dale Stephens,Solly March,Anthony Knockaert,Glenn Murray,Pascal Gross,2.36,3.14,3.31,2.35,1.59,11,8
7,6469,Liverpool,West Ham,2018,12:30,53235.0,2017-18,4,0,Alisson,Virgil van Dijk,Joe Gomez,Andy Robertson,Trent Alexander-Arnold,Georginio Wijnaldum,Naby Keita,James Milner,Roberto Firmino,Sadio Mané,Mohamed Salah,Lukasz Fabianski,Angelo Ogbonna,Fabián Balbuena,Arthur Masuaku,Ryan Fredericks,Jack Wilshere,Declan Rice,Mark Noble,Marko Arnautovic,Felipe Anderson,Michail Antonio,3.83,3.80,1.92,1.55,2.42,12,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,7714,Burnley,Crystal Palace,2021,15:00,18028.0,2020-21,3,3,Nick Pope,Ben Mee,James Tarkowski,Charlie Taylor,Matthew Lowton,Josh Brownhill,Ashley Westwood,Dwight McNeil,Johann Gudmundsson,Maxwel Cornet,Chris Wood,Vicente Guaita,Marc Guehi,Joachim Andersen,Tyrick Mitchell,Joel Ward,Luka Milivojevic,Cheikhou Kouyaté,Conor Gallagher,Christian Benteke,Wilfried Zaha,Jordan Ayew,2.77,3.23,2.70,2.18,1.70,20,11
1256,7718,Wolves,West Ham,2021,15:00,30667.0,2020-21,1,0,José Sá,Conor Coady,Romain Saïss,Max Kilman,João Moutinho,Rúben Neves,Rayan Aït-Nouri,Nélson Semedo,Raúl Jiménez,Hwang Hee-Chan,Daniel Podence,Lukasz Fabianski,Kurt Zouma,Craig Dawson,Aaron Cresswell,Ben Johnson,Saïd Benrahma,Declan Rice,Tomás Soucek,Michail Antonio,Pablo Fornals,Jarrod Bowen,3.02,3.27,2.47,2.06,1.79,20,11
1257,7719,Liverpool,Arsenal,2021,17:30,53092.0,2020-21,4,0,Alisson,Virgil van Dijk,Joël Matip,Konstantinos Tsimikas,Trent Alexander-Arnold,Fabinho,Thiago Alcántara,Alex Oxlade-Chamberlain,Diogo Jota,Sadio Mané,Mohamed Salah,Aaron Ramsdale,Gabriel,Ben White,Nuno Tavares,Takehiro Tomiyasu,Albert Sambi Lokonga,Thomas Partey,Emile Smith Rowe,Bukayo Saka,Pierre-Emerick Aubameyang,Alexandre Lacazette,1.48,4.77,6.38,1.49,2.64,20,11
1258,7720,Manchester City,Everton,2021,14:00,52571.0,2020-21,3,0,Ederson,Aymeric Laporte,John Stones,João Cancelo,Kyle Walker,Rodri,Ilkay Gündogan,Bernardo Silva,Cole Palmer,Phil Foden,Raheem Sterling,Jordan P

In [6]:
#Merging into table with team and opposition stats DONE
#Functions about top scorer,goalie and best player playing 
#Function about last 5 games goals and points DONE
#Function about last 10 games goals and points DONE

In [7]:
# cleaning matches data

from datetime import datetime

df = pd.DataFrame(matches)
df = df.rename(columns={
                    "home":"team", 
                    "away":"opponent", 
                    "time (utc)":"time", 
                    "home_score":"score", 
                    "away_score": "opp_score",
                    "AvgH":"win_odds",
                    "AvgD":"draw_odds",
                    "AvgA":"loss_odds",
                    "Avg>2.5":"over_2.5goals_odds",
                    "Avg<2.5":"under_2.5goals_odds"})
new = {}
for i in range(1, 12):
    new["home_starting_" + str(i)] = "starting_" + str(i)
for i in range(1, 12):
    new["away_starting_" + str(i)] = "opp_starting_" + str(i)
df = df.rename(columns=new)

# attendance has some weird NaNs that I couldn't get rid of so this doesn't work
df['attendance'] = df['attendance'].astype(int)

df.head()

,index,team,opponent,year,time,attendance,league,score,opp_score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month
0,6462,Manchester Utd,Leicester City,2018,19:00,74439,2017-18,2,1,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Matteo Darmian,Andreas Pereira,Paul Pogba,Fred,Marcus Rashford,Alexis Sánchez,Juan Mata,Kasper Schmeichel,Harry Maguire,Wes Morgan,Ben Chilwell,Daniel Amartey,James Maddison,Wilfred Ndidi,Adrien Silva,Kelechi Iheanacho,Demarai Gray,Ricardo Pereira,1.56,3.92,7.06,2.03,1.79,10,8
1,6463,Newcastle Utd,Tottenham,2018,11:30,51749,2017-18,1,2,Martin Dubravka,Ciaran Clark,Jamaal Lascelles,Paul Dummett,DeAndre Yedlin,Mohamed Diamé,Jonjo Shelvey,Kenedy,Matt Ritchie,Joselu,Ayoze Perez,Hugo Lloris,Jan Vertonghen,Davinson Sánchez,Ben Davies,Serge Aurier,Eric Dier,Dele Alli,Moussa Sissoko,Harry Kane,Christian Eriksen,Lucas Moura,1.88,3.53,4.37,1.98,1.83,11,8
4,6466,Huddersfield,Chelsea,2018,14:00,24121,2017-18,0,3,Ben Hamer,Christopher Schindler,Terence Kongolo,Mathias Jørgensen,Phillip Biling,Aaron Mooy,Jonathan Hogg,Chris Löwe,Florent Hadergjonaj,Steve Mounié,Alex Pritchard,Kepa Arrizabalaga,David Luiz,Antonio Rüdiger,Marcos Alonso,César Azpilicueta,Jorginho,Ross Barkley,N'Golo Kanté,Álvaro Morata,Willian,Pedro,3.83,3.40,2.05,2.01,1.81,11,8
5,6467,Watford,Brighton,2018,14:00,20051,2017-18,2,0,Ben Foster,Christian Kabasele,Craig Cathcart,José Holebas,Daryl Janmaat,Étienne Capoue,Abdoulaye Doucouré,Troy Deeney,Andre Gray,Roberto Pereyra,Will Hughes,Mat Ryan,Lewis Dunk,Shane Duffy,Bernardo,Bruno,Davy Propper,Dale Stephens,Solly March,Anthony Knockaert,Glenn Murray,Pascal Gross,2.36,3.14,3.31,2.35,1.59,11,8
7,6469,Liverpool,West Ham,2018,12:30,53235,2017-18,4,0,Alisson,Virgil van Dijk,Joe Gomez,Andy Robertson,Trent Alexander-Arnold,Georginio Wijnaldum,Naby Keita,James Milner,Roberto Firmino,Sadio Mané,Mohamed Salah,Lukasz Fabianski,Angelo Ogbonna,Fabián Balbuena,Arthur Masuaku,Ryan Fredericks,Jack Wilshere,Declan Rice,Mark Noble,Marko Arnautovic,Felipe Anderson,Michail Antonio,3.83,3.80,1.92,1.55,2.42,12,8


In [8]:
# prepare reverse matches and drop opp_score from matches

df2 = pd.DataFrame(df)
df = df.drop('opp_score', axis=1)
df2 = df2.drop('score', axis=1)
df2 = df2.rename(columns={"opp_score":"score"})

In [9]:
# finalize reverse matches

df2 = df2.rename(columns={"team":"opponent", 
                        "opponent":"team",
                         "win_odds":"loss_odds",
                         "loss_odds":"win_odds"})
new = {}
for i in range(1, 12):
    new["opp_starting_" + str(i)] = "starting_" + str(i)
for i in range(1, 12):
    new["starting_" + str(i)] = "opp_starting_" + str(i)
df2 = df2.rename(columns=new)

df2.head()

,index,opponent,team,year,time,attendance,league,score,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,loss_odds,draw_odds,win_odds,over_2.5goals_odds,under_2.5goals_odds,day,month
0,6462,Manchester Utd,Leicester City,2018,19:00,74439,2017-18,1,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Matteo Darmian,Andreas Pereira,Paul Pogba,Fred,Marcus Rashford,Alexis Sánchez,Juan Mata,Kasper Schmeichel,Harry Maguire,Wes Morgan,Ben Chilwell,Daniel Amartey,James Maddison,Wilfred Ndidi,Adrien Silva,Kelechi Iheanacho,Demarai Gray,Ricardo Pereira,1.56,3.92,7.06,2.03,1.79,10,8
1,6463,Newcastle Utd,Tottenham,2018,11:30,51749,2017-18,2,Martin Dubravka,Ciaran Clark,Jamaal Lascelles,Paul Dummett,DeAndre Yedlin,Mohamed Diamé,Jonjo Shelvey,Kenedy,Matt Ritchie,Joselu,Ayoze Perez,Hugo Lloris,Jan Vertonghen,Davinson Sánchez,Ben Davies,Serge Aurier,Eric Dier,Dele Alli,Moussa Sissoko,Harry Kane,Christian Eriksen,Lucas Moura,1.88,3.53,4.37,1.98,1.83,11,8
4,6466,Huddersfield,Chelsea,2018,14:00,24121,2017-18,3,Ben Hamer,Christopher Schindler,Terence Kongolo,Mathias Jørgensen,Phillip Biling,Aaron Mooy,Jonathan Hogg,Chris Löwe,Florent Hadergjonaj,Steve Mounié,Alex Pritchard,Kepa Arrizabalaga,David Luiz,Antonio Rüdiger,Marcos Alonso,César Azpilicueta,Jorginho,Ross Barkley,N'Golo Kanté,Álvaro Morata,Willian,Pedro,3.83,3.40,2.05,2.01,1.81,11,8
5,6467,Watford,Brighton,2018,14:00,20051,2017-18,0,Ben Foster,Christian Kabasele,Craig Cathcart,José Holebas,Daryl Janmaat,Étienne Capoue,Abdoulaye Doucouré,Troy Deeney,Andre Gray,Roberto Pereyra,Will Hughes,Mat Ryan,Lewis Dunk,Shane Duffy,Bernardo,Bruno,Davy Propper,Dale Stephens,Solly March,Anthony Knockaert,Glenn Murray,Pascal Gross,2.36,3.14,3.31,2.35,1.59,11,8
7,6469,Liverpool,West Ham,2018,12:30,53235,2017-18,0,Alisson,Virgil van Dijk,Joe Gomez,Andy Robertson,Trent Alexander-Arnold,Georginio Wijnaldum,Naby Keita,James Milner,Roberto Firmino,Sadio Mané,Mohamed Salah,Lukasz Fabianski,Angelo Ogbonna,Fabián Balbuena,Arthur Masuaku,Ryan Fredericks,Jack Wilshere,Declan Rice,Mark Noble,Marko Arnautovic,Felipe Anderson,Michail Antonio,3.83,3.80,1.92,1.55,2.42,12,8


In [10]:
# combine matches and reversed matches

combined_df = df.append(df2)
combined_df.index = range(len(combined_df.index))
combined_df[(combined_df.attendance == "74439")]

,index,team,opponent,year,time,attendance,league,score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month


In [11]:
# merge matches and teams' stats

pre_final = pd.merge(combined_df, full_teamtable, on=['team', 'league'])
pre_final.head()

,index,team,opponent,year,time,attendance,league,score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month,Rk,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Goalkeeper,# Pl,Age,Poss,Gls,Ast,G-PK,PK,PKatt,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Best Player,Top Scorer,Top Scorer Goals
0,6462,Manchester Utd,Leicester City,2018,19:00,74439,2017-18,2,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Matteo Darmian,Andreas Pereira,Paul Pogba,Fred,Marcus Rashford,Alexis Sánchez,Juan Mata,Kasper Schmeichel,Harry Maguire,Wes Morgan,Ben Chilwell,Daniel Amartey,James Maddison,Wilfred Ndidi,Adrien Silva,Kelechi Iheanacho,Demarai Gray,Ricardo Pereira,1.56,3.92,7.06,2.03,1.79,10,8,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.5,David de Gea,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,60.7,58.2,43.8,102.1,1.6,1.15,2.75,1.53,2.69,26,18,25,1,1,69,1,0.68,0.47,1.16,0.66,1.13,41.7,41.0,29.8,70.8,1.1,0.79,1.88,1.08,1.86,513,176,34.3,13.5,4.63,0.13,0.38,17.6,22,0.12,6.3,7.8,David de Gea,Romelu Lukaku,16
1,6491,Manchester Utd,Tottenham,2018,19:00,744,2017-18,0,David de Gea,Chris Smalling,Phil Jones,Ander Herrera,Fred,Paul Pogba,Nemanja Matic,Luke Shaw,Antonio Valencia,Romelu Lukaku,Jesse Lingard,Hugo Lloris,Jan Vertonghen,Toby Alderweireld,Mousa Dembélé,Danny Rose,Kieran Trippier,Christian Eriksen,Dele Alli,Eric Dier,Harry Kane,Lucas Moura,2.56,3.27,2.86,2.03,1.79,27,8,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.5,David de Gea,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,60.7,58.2,43.8,102.1,1.6,1.15,2.75,1.53,2.69,26,18,25,1,1,69,1,0.68,0.47,1.16,0.66,1.13,41.7,41.0,29.8,70.8,1.1,0.79,1.88,1.08,1.86,513,176,34.3,13.5,4.63,0.13,0.38,17.6,22,0.12,6.3,7.8,David de Gea,Romelu Lukaku,16
2,6538,Manchester Utd,Newcastle Utd,2018,16:30,74519,2017-18,3,David de Gea,Chris Smalling,Eric Bailly,Luke Shaw,Ashley Young,Nemanja Matic,Paul Pogba,Scott McTominay,Romelu Lukaku,Anthony Martial,Marcus Rashford,Martin Dubravka,Federico Fernández,Jamaal Lascelles,Javier Manquillo,DeAndre Yedlin,Mohamed Diamé,Jonjo Shelvey,Kenedy,Matt Ritchie,Yoshinori Muto,Ayoze Perez,2.13,3.55,3.45,1.77,2.06,6,10,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.5,David de Gea,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,60.7,58.2,43.8,102.1,1.6,1.15,2.75,1.53,2.69,26,18,25,1,1,69,1,0.68,0.47,1.16,0.66,1.13,41.7,41.0,29.8,70.8,1.1,0.79,1.88,1.08,1.86,513,176,34.3,13.5,4.63,0.13,0.38,17.6,22,0.12,6.3,7.8,David de Gea,Romelu Lukaku,16
3,6560,Manchester Utd,Everton,2018,16:00,74525,2017-18,2,David de Gea,Victor Lindelöf,Chris Smalling,Luke Shaw,Ashley Young,Nemanja Matic,Paul Pogba,Fred,Marcus Rashford,Anthony Martial,Juan Mata,Jordan Pickford,Kurt Zouma,Michael Keane,Lucas Digne,Seamus Coleman,Gylfi Sigurdsson,Idrissa Gueye,André Gomes,Richarlison,Bernard,Theo Walcott,1.73,3.80,4.93,1.81,2.01,28,10,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.5,David de Gea,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,60.7,58.2,43.8,102.1,1.6,1.15,2.75,1.53,2.69,26,18,25,1,1,69,1,0.68,0.47,1.16,0.66,1.13,41.7,41.0,29.8,70.8,1.1,0.79,1.88,1.08,1.86,513,176,34.3,13.5,4.63,0.13,0.38,17.6,22,0.12,6.3,7.8,David de Gea,Romelu Lukaku,16
4,6585,Manchester Utd,Crystal Palace,2018,15:00,74516,2017-18,0,David de Gea,David de Gea,Victor Lindelöf,Chris Smalling,Ashley Young,Matteo Darmian,Nemanja Matic,Paul Pogba,Jesse Lingard,Romelu Lukaku,Anthony Martial,Wayne Hennessey,Wayne Hennessey,Mamadou Sakho,James Tomkins,Patrick van Aanholt,Aaron Wan-Bissaka,Luka Milivojevic,Cheikhou Kouyaté,Max Meyer,James McArt

In [12]:
# create teamtable with alternative column names

opp_full_teamtable = pd.DataFrame(full_teamtable)
new = {}
for col in opp_full_teamtable.columns:
    new[str(col)] = "opp_" + str(col)
opp_full_teamtable = opp_full_teamtable.rename(columns=new)

opp_full_teamtable = opp_full_teamtable.rename(columns={"opp_team":"opponent", "opp_league":"league"})
opp_full_teamtable.head()

,opp_Rk,opponent,opp_W,opp_D,opp_L,opp_GF,opp_GA,opp_GD,opp_Pts,opp_xG,opp_xGA,opp_xGD,opp_xGD/90,opp_Goalkeeper,opp_# Pl,opp_Age,opp_Poss,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_xG,opp_npxG,opp_xA,opp_npxG+xA,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_CrdY,opp_CrdR,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_xG,opp_npxG,opp_xA,opp_npxG+xA,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Sh,opp_SoT,opp_SoT%,opp_Sh/90,opp_SoT/90,opp_G/Sh,opp_G/SoT,opp_Dist,opp_FK,opp_npxG/Sh,opp_G-xG,opp_np:G-xG,league,opp_Best Player,opp_Top Scorer,opp_Top Scorer Goals
0,6,Arsenal,19,6,13,74,51,23,63,68.4,50.6,17.8,0.47,Petr Čech,30,26.8,61.5,73,61,69,4,5,1.92,1.61,3.53,1.82,3.42,68.4,64.6,51.3,115.9,1.80,1.35,3.15,1.70,3.05,51,38,46,5,6,52,4,1.34,1.00,2.34,1.21,2.21,50.6,46.1,34.0,80.1,1.33,0.89,2.23,1.21,2.11,590,227,38.5,15.53,5.97,0.12,0.30,17.4,30,0.11,4.6,4.4,2017-18,Aaron Ramsey,Alexandre Lacazette,14
1,12,Bournemouth,11,11,16,45,61,-16,44,44.8,61.4,-16.6,-0.44,Asmir Begović,22,26.9,47.8,45,36,42,3,3,1.18,0.95,2.13,1.11,2.05,44.8,42.6,32.2,74.8,1.18,0.85,2.03,1.12,1.97,60,44,58,2,3,73,1,1.58,1.16,2.74,1.53,2.68,61.4,59.3,43.7,103.0,1.62,1.15,2.77,1.56,2.71,473,153,32.3,12.45,4.03,0.09,0.27,17.5,13,0.09,0.2,-0.6,2017-18,Nathan Ake,Callum Wilson Joshua King,8
2,15,Brighton,9,13,16,34,54,-20,40,37.2,52.6,-15.5,-0.41,Mathew Ryan,24,27.5,43.7,33,25,28,5,7,0.87,0.66,1.53,0.74,1.39,37.2,31.8,23.0,54.8,0.98,0.60,1.58,0.84,1.44,49,36,48,1,3,30,2,1.29,0.95,2.24,1.26,2.21,52.6,50.3,34.5,84.9,1.38,0.91,2.29,1.32,2.23,381,113,29.7,10.03,2.97,0.07,0.25,17.9,3,0.09,-4.2,-3.8,2017-18,Pascal Groß,Glenn Murray,12
3,7,Burnley,14,12,12,36,39,-3,54,34.5,51.7,-17.3,-0.45,Nick Pope,24,27.5,43.9,35,26,35,0,0,0.92,0.68,1.61,0.92,1.61,34.5,34.5,23.8,58.3,0.91,0.63,1.53,0.91,1.53,38,29,35,3,5,45,6,1.00,0.76,1.76,0.92,1.68,51.7,48.2,37.4,85.7,1.36,0.99,2.35,1.27,2.25,387,126,32.6,10.18,3.32,0.09,0.28,16.0,16,0.09,0.5,0.5,2017-18,Nick Pope,Chris Wood,10
4,5,Chelsea,21,7,10,62,38,24,70,57.9,35.3,22.7,0.60,Thibaut Courtois,26,26.7,55.4,60,42,57,3,3,1.58,1.11,2.68,1.50,2.61,57.9,55.7,42.2,97.9,1.52,1.11,2.64,1.46,2.58,36,26,34,2,2,75,1,0.95,0.68,1.63,0.89,1.58,35.3,33.8,25.4,59.1,0.93,0.67,1.60,0.89,1.56,602,209,34.7,15.84,5.50,0.09,0.27,17.4,19,0.09,2.1,1.3,2017-18,N'Golo Kanté,Eden Hazard,12


In [13]:
# merge matches and oppenents' stats

final = pd.merge(pre_final, opp_full_teamtable, on=['opponent', 'league'])
final.head()

,index,team,opponent,year,time,attendance,league,score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month,Rk,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Goalkeeper,# Pl,Age,Poss,Gls,Ast,G-PK,PK,PKatt,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Best Player,Top Scorer,Top Scorer Goals,opp_Rk,opp_W,opp_D,opp_L,opp_GF,opp_GA,opp_GD,opp_Pts,opp_xG,opp_xGA,opp_xGD,opp_xGD/90,opp_Goalkeeper,opp_# Pl,opp_Age,opp_Poss,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_xG,opp_npxG,opp_xA,opp_npxG+xA,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_CrdY,opp_CrdR,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_xG,opp_npxG,opp_xA,opp_npxG+xA,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Sh,opp_SoT,opp_SoT%,opp_Sh/90,opp_SoT/90,opp_G/Sh,opp_G/SoT,opp_Dist,opp_FK,opp_npxG/Sh,opp_G-xG,opp_np:G-xG,opp_Best Player,opp_Top Scorer,opp_Top Scorer Goals
0,6462,Manchester Utd,Leicester City,2018,19:00,74439,2017-18,2,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Matteo Darmian,Andreas Pereira,Paul Pogba,Fred,Marcus Rashford,Alexis Sánchez,Juan Mata,Kasper Schmeichel,Harry Maguire,Wes Morgan,Ben Chilwell,Daniel Amartey,James Maddison,Wilfred Ndidi,Adrien Silva,Kelechi Iheanacho,Demarai Gray,Ricardo Pereira,1.56,3.92,7.06,2.03,1.79,10,8,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.50,David de Gea,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,60.7,58.2,43.8,102.1,1.60,1.15,2.75,1.53,2.69,26,18,25,1,1,69,1,0.68,0.47,1.16,0.66,1.13,41.7,41.0,29.8,70.8,1.10,0.79,1.88,1.08,1.86,513,176,34.3,13.50,4.63,0.13,0.38,17.6,22,0.12,6.3,7.8,David de Gea,Romelu Lukaku,16,9,12,11,15,56,60,-4,47,48.1,49.9,-1.8,-0.05,Kasper Schmeichel,27,27.2,47.9,53,38,48,5,6,1.39,1.0,2.39,1.26,2.26,48.1,43.6,31.9,75.5,1.27,0.84,2.11,1.15,1.99,58,47,56,2,4,52,1,1.53,1.24,2.76,1.47,2.71,49.9,46.9,37.1,83.9,1.31,0.98,2.29,1.23,2.21,418,137,32.8,11.0,3.61,0.11,0.35,17.0,12,0.11,4.9,4.4,Harry Maguire,Jamie Vardy,20
1,6709,Manchester Utd,Leicester City,2019,14:05,32148,2017-18,1,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Ashley Young,Nemanja Matic,Ander Herrera,Alexis Sánchez,Marcus Rashford,Paul Pogba,Jesse Lingard,Kasper Schmeichel,Harry Maguire,Jonny Evans,Ben Chilwell,Ricardo Pereira,James Maddison,Wilfred Ndidi,Nampalys Mendy,Jamie Vardy,Harvey Barnes,Demarai Gray,2.11,3.41,3.63,1.81,2.02,3,2,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.50,David de Gea,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,60.7,58.2,43.8,102.1,1.60,1.15,2.75,1.53,2.69,26,18,25,1,1,69,1,0.68,0.47,1.16,0.66,1.13,41.7,41.0,29.8,70.8,1.10,0.79,1.88,1.08,1.86,513,176,34.3,13.50,4.63,0.13,0.38,17.6,22,0.12,6.3,7.8,David de Gea,Romelu Lukaku,16,9,12,11,15,56,60,-4,47,48.1,49.9,-1.8,-0.05,Kasper Schmeichel,27,27.2,47.9,53,38,48,5,6,1.39,1.0,2.39,1.26,2.26,48.1,43.6,31.9,75.5,1.27,0.84,2.11,1.15,1.99,58,47,56,2,4,52,1,1.53,1.24,2.76,1.47,2.71,49.9,46.9,37.1,83.9,1.31,0.98,2.29,1.23,2.21,418,137,32.8,11.0,3.61,0.11,0.35,17.0,12,0.11,4.9,4.4,Harry Maguire,Jamie Vardy,20
2,6527,Newcastle Utd,Leicester City,2018,14:00,51523,2017-18,0,Martin Dubravka,Ciaran Clark,Jamaal Lascelles,Kenedy,DeAndre Yedlin,Jonjo Shelvey,Mohamed Diamé,Christian Atsu,Matt Ritchie,Joselu,Ayoze Perez,Kasper Schmeichel,Harry Maguire,Wes Morgan,Ben Chilwell,Daniel Amartey,Kelechi Iheanacho,Wilfred Ndidi,Nampalys Mendy,Jamie Vardy,James Maddison,Ricardo Pereira,2.56,3.22,2.93,2.20,1.68,29,9,10,12,8,18,39,47,-8,44,45.

In [14]:
#function which is used to convert the whole dataframe to datetime
def toDatetime(x):
    year = str(x.year)
    time = str(x.time)
    month = str(x.month) if x.month > 9 else "0"+str(x.month)
    day = str(x.day) if x.day > 9 else "0"+str(x.day)
    return datetime.fromisoformat(year+"-"+month+"-"+day+"T"+time)
    

In [15]:
matches["datetime"] = matches[["time","year","day","month"]].apply(toDatetime, axis=1)
matches.drop(["time","year","day","month"], axis=1)

,index,home,away,attendance,league,home_score,away_score,home_starting_1,home_starting_2,home_starting_3,home_starting_4,home_starting_5,home_starting_6,home_starting_7,home_starting_8,home_starting_9,home_starting_10,home_starting_11,away_starting_1,away_starting_2,away_starting_3,away_starting_4,away_starting_5,away_starting_6,away_starting_7,away_starting_8,away_starting_9,away_starting_10,away_starting_11,AvgH,AvgD,AvgA,Avg>2.5,Avg<2.5,datetime
0,6462,Manchester Utd,Leicester City,74439.0,2017-18,2,1,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Matteo Darmian,Andreas Pereira,Paul Pogba,Fred,Marcus Rashford,Alexis Sánchez,Juan Mata,Kasper Schmeichel,Harry Maguire,Wes Morgan,Ben Chilwell,Daniel Amartey,James Maddison,Wilfred Ndidi,Adrien Silva,Kelechi Iheanacho,Demarai Gray,Ricardo Pereira,1.56,3.92,7.06,2.03,1.79,2018-08-10 19:00:00
1,6463,Newcastle Utd,Tottenham,51749.0,2017-18,1,2,Martin Dubravka,Ciaran Clark,Jamaal Lascelles,Paul Dummett,DeAndre Yedlin,Mohamed Diamé,Jonjo Shelvey,Kenedy,Matt Ritchie,Joselu,Ayoze Perez,Hugo Lloris,Jan Vertonghen,Davinson Sánchez,Ben Davies,Serge Aurier,Eric Dier,Dele Alli,Moussa Sissoko,Harry Kane,Christian Eriksen,Lucas Moura,1.88,3.53,4.37,1.98,1.83,2018-08-11 11:30:00
4,6466,Huddersfield,Chelsea,24121.0,2017-18,0,3,Ben Hamer,Christopher Schindler,Terence Kongolo,Mathias Jørgensen,Phillip Biling,Aaron Mooy,Jonathan Hogg,Chris Löwe,Florent Hadergjonaj,Steve Mounié,Alex Pritchard,Kepa Arrizabalaga,David Luiz,Antonio Rüdiger,Marcos Alonso,César Azpilicueta,Jorginho,Ross Barkley,N'Golo Kanté,Álvaro Morata,Willian,Pedro,3.83,3.40,2.05,2.01,1.81,2018-08-11 14:00:00
5,6467,Watford,Brighton,20051.0,2017-18,2,0,Ben Foster,Christian Kabasele,Craig Cathcart,José Holebas,Daryl Janmaat,Étienne Capoue,Abdoulaye Doucouré,Troy Deeney,Andre Gray,Roberto Pereyra,Will Hughes,Mat Ryan,Lewis Dunk,Shane Duffy,Bernardo,Bruno,Davy Propper,Dale Stephens,Solly March,Anthony Knockaert,Glenn Murray,Pascal Gross,2.36,3.14,3.31,2.35,1.59,2018-08-11 14:00:00
7,6469,Liverpool,West Ham,53235.0,2017-18,4,0,Alisson,Virgil van Dijk,Joe Gomez,Andy Robertson,Trent Alexander-Arnold,Georginio Wijnaldum,Naby Keita,James Milner,Roberto Firmino,Sadio Mané,Mohamed Salah,Lukasz Fabianski,Angelo Ogbonna,Fabián Balbuena,Arthur Masuaku,Ryan Fredericks,Jack Wilshere,Declan Rice,Mark Noble,Marko Arnautovic,Felipe Anderson,Michail Antonio,3.83,3.80,1.92,1.55,2.42,2018-08-12 12:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,7714,Burnley,Crystal Palace,18028.0,2020-21,3,3,Nick Pope,Ben Mee,James Tarkowski,Charlie Taylor,Matthew Lowton,Josh Brownhill,Ashley Westwood,Dwight McNeil,Johann Gudmundsson,Maxwel Cornet,Chris Wood,Vicente Guaita,Marc Guehi,Joachim Andersen,Tyrick Mitchell,Joel Ward,Luka Milivojevic,Cheikhou Kouyaté,Conor Gallagher,Christian Benteke,Wilfried Zaha,Jordan Ayew,2.77,3.23,2.70,2.18,1.70,2021-11-20 15:00:00
1256,7718,Wolves,West Ham,30667.0,2020-21,1,0,José Sá,Conor Coady,Romain Saïss,Max Kilman,João Moutinho,Rúben Neves,Rayan Aït-Nouri,Nélson Semedo,Raúl Jiménez,Hwang Hee-Chan,Daniel Podence,Lukasz Fabianski,Kurt Zouma,Craig Dawson,Aaron Cresswell,Ben Johnson,Saïd Benrahma,Declan Rice,Tomás Soucek,Michail Antonio,Pablo Fornals,Jarrod Bowen,3.02,3.27,2.47,2.06,1.79,2021-11-20 15:00:00
1257,7719,Liverpool,Arsenal,53092.0,2020-21,4,0,Alisson,Virgil van Dijk,Joël Matip,Konstantinos Tsimikas,Trent Alexander-Arnold,Fabinho,Thiago Alcántara,Alex Oxlade-Chamberlain,Diogo Jota,Sadio Mané,Mohamed Salah,Aaron Ramsdale,Gabriel,Ben White,Nuno Tavares,Takehiro Tomiyasu,Albert Sambi Lokonga,Thomas Partey,Emile Smith Rowe,Bukayo Saka,Pierre-Emerick Aubameyang,Alexandre Lacazette,1.48,4.77,6.38,1.49,2.64,2021-11-20 17:30:00
1258,7720,Manchester City,Everton,52571.0,2020-21,3,0,Ederson,Aymeric Laporte,John Stones,João Cancelo,Kyle Walker,Rodri,Ilkay Gündogan,Bernardo Silva,Cole Palmer,Phil Foden,Raheem Sterling,Jordan Pickfo

In [16]:
#Function about last n games goals and points
#returns an array with the structure: [wins, losses, ties, team_goals, opp_goals]
def FindLastGames(n, team, season, datetime):
    results = matches[((matches['home'] == team) | (matches['away'] == team)) & (matches['league'] == season) & (matches['datetime'] < datetime)]
    results.sort_values("datetime", ascending=True)
    display(results)
    
    #if the results are
    if len(results)<n:
        n = len(results)
    
    results = results.tail(n)
    
    #structure - [wins, losses, ties, team_goals, opp_goals]
    out = [0,0,0,0,0] 
    
    for i in range(n):
        if results.iloc[i]["away"]==team:
            team = results.iloc[i]["away_score"]
            opponent = results.iloc[i]["home_score"]
        else:
            team = results.iloc[i]["home_score"]
            opponent = results.iloc[i]["away_score"]
        out[3]+=team
        out[4]+=opponent
        
        if(team>opponent):
            out[0]+=1
        elif(team==opponent):
            out[2]+=1
        else:
            out[1]+=1
    
    return out
            
FindLastGames(10, "Manchester Utd", "2020-21", datetime.fromisoformat("2021-10-30T16:30"))

,index,home,away,year,time,attendance,league,home_score,away_score,home_starting_1,home_starting_2,home_starting_3,home_starting_4,home_starting_5,home_starting_6,home_starting_7,home_starting_8,home_starting_9,home_starting_10,home_starting_11,away_starting_1,away_starting_2,away_starting_3,away_starting_4,away_starting_5,away_starting_6,away_starting_7,away_starting_8,away_starting_9,away_starting_10,away_starting_11,AvgH,AvgD,AvgA,Avg>2.5,Avg<2.5,day,month,datetime
1141,7603,Manchester Utd,Leeds United,2021,11:30,72732.0,2020-21,5,1,David de Gea,Harry Maguire,Victor Lindelöf,Luke Shaw,Aaron Wan-Bissaka,Bruno Fernandes,Fred,Scott McTominay,Mason Greenwood,Paul Pogba,Daniel James,Illan Meslier,Liam Cooper,Pascal Struijk,Robin Koch,Stuart Dallas,Luke Ayling,Mateusz Klich,Rodrigo,Jack Harrison,Raphinha,Patrick Bamford,1.55,4.48,5.87,1.65,2.29,14,8,2021-08-14 11:30:00
1156,7618,Southampton,Manchester Utd,2021,13:00,0.0,2020-21,1,1,Alex McCarthy,Mohammed Salisu,Jack Stephens,Romain Perraud,Valentino Livramento,James Ward-Prowse,Oriol Romeu,Moussa Djenepo,Theo Walcott,Adam Armstrong,Ché Adams,David de Gea,Harry Maguire,Victor Lindelöf,Luke Shaw,Aaron Wan-Bissaka,Bruno Fernandes,Nemanja Matic,Fred,Anthony Martial,Paul Pogba,Mason Greenwood,5.36,4.26,1.61,1.72,2.16,22,8,2021-08-22 13:00:00
1169,7631,Wolves,Manchester Utd,2021,15:30,30621.0,2020-21,0,1,José Sá,Conor Coady,Romain Saïss,Max Kilman,João Moutinho,Rúben Neves,Fernando Marçal,Nélson Semedo,Raúl Jiménez,Adama Traoré,Francisco Trincão,David de Gea,Harry Maguire,Raphaël Varane,Luke Shaw,Aaron Wan-Bissaka,Bruno Fernandes,Fred,Paul Pogba,Mason Greenwood,Jadon Sancho,Daniel James,4.91,3.48,1.82,2.01,1.83,29,8,2021-08-29 15:30:00
1174,7636,Manchester Utd,Newcastle Utd,2021,14:00,72732.0,2020-21,4,1,David de Gea,Harry Maguire,Raphaël Varane,Luke Shaw,Aaron Wan-Bissaka,Bruno Fernandes,Nemanja Matic,Paul Pogba,Cristiano Ronaldo,Jadon Sancho,Mason Greenwood,Freddie Woodman,Jamaal Lascelles,Ciaran Clark,Isaac Hayden,Matt Ritchie,Javier Manquillo,Sean Longstaff,Joe Willock,Joelinton,Miguel Almirón,Allan Saint-Maximin,1.20,7.27,14.77,1.44,2.79,11,9,2021-09-11 14:00:00
1188,7650,West Ham,Manchester Utd,2021,13:00,0.0,2020-21,1,2,Lukasz Fabianski,Angelo Ogbonna,Kurt Zouma,Aaron Cresswell,Vladimír Coufal,Declan Rice,Tomás Soucek,Nikola Vlasic,Pablo Fornals,Saïd Benrahma,Jarrod Bowen,David de Gea,Harry Maguire,Raphaël Varane,Luke Shaw,Aaron Wan-Bissaka,Bruno Fernandes,Fred,Scott McTominay,Cristiano Ronaldo,Paul Pogba,Mason Greenwood,4.51,3.90,1.78,1.69,2.21,19,9,2021-09-19 13:00:00
1191,7653,Manchester Utd,Aston Villa,2021,11:30,72922.0,2020-21,0,1,David de Gea,Harry Maguire,Raphaël Varane,Luke Shaw,Aaron Wan-Bissaka,Bruno Fernandes,Fred,Scott McTominay,Cristiano Ronaldo,Paul Pogba,Mason Greenwood,Emiliano Martínez,Kortney Hause,Tyrone Mings,Ezri Konsa,Douglas Luiz,John McGinn,Jacob Ramsey,Matt Targett,Matthew Cash,Ollie Watkins,Danny Ings,1.40,5.08,7.84,1.60,2.39,25,9,2021-09-25 11:30:00
1200,7662,Manchester Utd,Everton,2021,11:30,73128.0,2020-21,1,1,David de Gea,Raphaël Varane,Victor Lindelöf,Luke Shaw,Aaron Wan-Bissaka,Bruno Fernandes,Fred,Scott McTominay,Edinson Cavani,Anthony Martial,Mason Greenwood,Jordan Pickford,Michael Keane,Yerry Mina,Lucas Digne,Ben Godfrey,Allan,Abdoulaye Doucouré,Anthony Gordon,Andros Townsend,Salomón Rondón,Demarai Gray,1.50,4.38,7.08,1.81,2.05,2,10,2021-10-02 11:30:00
1212,7674,Leicester City,Manchester Utd,2021,14:00,32219.0,2020-21,4,2,Kasper Schmeichel,Jonny Evans,Caglar Söyüncü,Daniel Amartey,Boubakary Soumaré,Youri Tielemans,James Maddison,Timothy Castagne,Ricardo Pereira,Jamie Vardy,Kelechi Iheanacho,David de Gea,Harry Maguire,Victor Lindelöf,Luke Shaw,Aaron Wan-Bissaka,Bruno Fernandes,Nemanja Matic,Paul Pogba,Cristiano Ronaldo,Jadon Sancho,Mason Greenwood,3.49,3.59,2.10,1.77,2.11,16,10,2021-10-16 14:00:00
1229,7691,Manchester Utd,Liverpool,2021,15:30,73088.0,2020-21,0,5,David de Gea,Harry Maguire,Victor Lindelöf,Luke Shaw,Aaron Wan-Bissaka,Bruno Fernan

[3, 4, 2, 16, 15]

In [17]:
#Functions about top scorer, goalie and best player playing 
def getTopScorer(team, season):
    results = full_teamtable[((full_teamtable['team'] == team) & (full_teamtable['league'] == season))]["Top Scorer"]
    return results[0]
    
def getGoalie(team, season):
    results = full_teamtable[((full_teamtable['team'] == team) & (full_teamtable['league'] == season))]["Goalkeeper"]
    return results[0]
    
def getBestPlayer(team, season):
    results = full_teamtable[((full_teamtable['team'] == team) & (full_teamtable['league'] == season))]["Best Player"]
    return results[0]


print(getTopScorer("Arsenal", "2017-18"))
print(getGoalie("Arsenal", "2017-18"))
print(getBestPlayer("Arsenal", "2017-18"))

Alexandre Lacazette
Petr Čech
Aaron Ramsey
